In [28]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from datetime import datetime

In [29]:
def distance(xml_path):
    df = pd.DataFrame(columns=["name", "time", "root_distnace"]) 
    for path, dir, files in os.walk(xml_path):
        for file in files:
            if os.path.splitext(file)[-1] == ".xml" and os.path.getsize(os.path.join(path, file)) != None:
                # Parse the xml file
                tree = ET.parse(os.path.join(path, file))
                root = tree.getroot()
                for image in root.iter("image"):
                    distance = 0
                    thickness = None
                    # Define the points of the polyline
                    root_points = list()
                    for polyline in image.iter("polyline"):
                        if polyline.get("label") == "root":
                            root_points.append(polyline.get("points"))
                    for root_point in root_points:
                        s_root_points = str(root_point).split(";")
                        xy_coord = list()
                        for s_root_point in s_root_points:
                            x_coord = int(float((str(s_root_point).split(",")[0])))
                            y_coord = int(float((str(s_root_point).split(",")[1])))
                            xy_coord.append([x_coord, y_coord])
                        # Create a LineString object from the points
                        polyline = LineString(xy_coord)

                        # Compute the length of the polyline
                        distance += polyline.length
                        thickness = distance / len(root_points)
                            
                    # Add a new row to the dataframe with the image name and length
                    s = image.get("name")
                    root_name = s.split('_')[0]
                    date_string = s.split('_')[-1].split(".")[0]
                    date_time = datetime.strptime(date_string, "%y%m%d%H%M%S")
                    formatted_date_time = date_time.strftime("%Y-%m-%d %H:%M:%S")
                    df = df.append({"name": root_name, "time": formatted_date_time, "root_distance": distance, 'root_thickness': thickness}, ignore_index=True)
    return df                

In [30]:
def main():
    xml_path = r'C:/Users/user/Desktop/capstone/data/01_xml'
    df = distance(xml_path)
    df.to_excel('temporary_root_distance_thickness_first.xlsx', index=False)

In [31]:
main()